In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


ruta = 'C:/Users/Sistemas/Desktop/pallets_cruzados_nuevo/scripts/definitivo_trabajo.xlsx'

df = pd.read_excel(ruta)

df.head()

,id_socio,socio,fecha,pallets,base,Promedio pallets por socio diario,Porcentaje pallets por socio,Parte variable,Total final por socio,Exceso Palets contra (Fijo+Variable),Exceso contra Fijo,cantidad_pallets_sacados,stock_ayer,ingresos_estimados,rotacion_diaria,promedio_rotacion_mensual
0,3.0,DOZ CARLOS,2025-03-01,24,10,19,3.00%,9,19,0,9,3,NaN,NaN,NaN,0.076762
1,3.0,DOZ CARLOS,2025-03-05,20,10,19,3.00%,9,19,0,9,3,24.0,-1.0,0.125000,0.076762
2,3.0,DOZ CARLOS,2025-03-06,19,10,19,3.00%,9,19,0,9,0,20.0,-1.0,0.000000,0.076762
3,3.0,DOZ CARLOS,2025-03-07,24,10,19,3.00%,9,19,0,9,2,19.0,7.0,0.105263,0.076762
4,3.0,DOZ CARLOS,2025-03-08,20,10,19,3.00%,9,19,0,9,0,24.0,-4.0,0.000000,0.076762


In [2]:
# Elimina espacios y normaliza (ej: 'Estado    ' -> 'Estado')
df.columns = df.columns.str.strip()  # Quita espacios
# Opcional: convertir a minúsculas (para uniformidad)
df.columns = df.columns.str.lower()  # 'Estado' -> 'estado'
print("Columnas disponibles:", df.columns.tolist())

Columnas disponibles: ['id_socio', 'socio', 'fecha', 'pallets', 'base', 'promedio pallets por socio diario', 'porcentaje pallets por socio', 'parte variable', 'total final por socio', 'exceso palets contra (fijo+variable)', 'exceso contra fijo', 'cantidad_pallets_sacados', 'stock_ayer', 'ingresos_estimados', 'rotacion_diaria', 'promedio_rotacion_mensual']


In [3]:
df.rename(columns={'promedio pallets por socio diario':'promedio_pallets_por_socio_diario','parte variable': 'parte_variable', 'total final por socio': 'total_final_por_socio', 'porcentaje pallets por socio': 'porcentaje_pallets_por_socio', 'promedio pallets por socio': 'promedio_pallets_por_socio', 'exceso palets contra (fijo+variable)': 'exceso_pallets_contra_fijo_+_variable', 'exceso contra fijo': 'exceso_contra_fijo'}, inplace=True)

In [4]:
print("Columnas disponibles:", df.columns.tolist())

Columnas disponibles: ['id_socio', 'socio', 'fecha', 'pallets', 'base', 'promedio_pallets_por_socio_diario', 'porcentaje_pallets_por_socio', 'parte_variable', 'total_final_por_socio', 'exceso_pallets_contra_fijo_+_variable', 'exceso_contra_fijo', 'cantidad_pallets_sacados', 'stock_ayer', 'ingresos_estimados', 'rotacion_diaria', 'promedio_rotacion_mensual']


In [5]:
# Asegurar que 'fecha' esté en formato datetime
df['fecha'] = pd.to_datetime(df['fecha'])

In [6]:
# Filtrar marzo
df_marzo = df[df['fecha'].dt.month == 3].copy()

In [7]:
# Crear caso base manual del 28 de febrero
caso_base_febrero = pd.DataFrame({
    'id_socio': [3, 5, 8, 9, 10, 11.111, 12, 13, 14, 16, 17.39, 19, 24, 25.63, 29, 32, 38, 53, 59, 60, 66, 68, 73, 76, 77, 79, 83, 119, 122, 128, 130, 135, 136],
    'socio': np.nan,  # se puede completar luego si se desea
    'fecha': pd.to_datetime('2025-02-28'),
    'pallets': [31, 27, 23, 9, 14, 116, 43, 10, 39, 3, 59, 21, 38, 34, 48, 19, 5, 15, 30, 5, 8, 46, 30, 44, 17, 11, 21, 27, 5, 26, 23, 18, 17],
    'cantidad_pallets_sacados': 0
})

# Unir febrero + marzo
df_completo = pd.concat([caso_base_febrero, df_marzo], ignore_index=True)

In [8]:
# Ordenar por socio y fecha
df_completo = df_completo.sort_values(by=['id_socio', 'fecha']).reset_index(drop=True)

In [9]:
# Calcular stock_ayer
df_completo['stock_ayer'] = df_completo.groupby('id_socio')['pallets'].shift(1)

In [10]:
# Calcular cantidad_pallets_sacados automáticamente
df_completo['cantidad_pallets_sacados_calc'] = (
    df_completo['stock_ayer'] + 0  # para dejarlo explícito
    - df_completo['pallets']
)

In [11]:
# Usar la columna calculada si lo deseás, o comparar
# Calcular ingresos estimados (es lo que se agregó en el día)
df_completo['ingresos_estimados'] = df_completo['pallets'] - (
    df_completo['stock_ayer'] - df_completo['cantidad_pallets_sacados']
)


In [12]:
# Calcular rotación diaria
df_completo['rotacion_diaria'] = df_completo['cantidad_pallets_sacados'] / df_completo['stock_ayer']


In [13]:
# Filtrar nuevamente marzo, ya con stock_ayer válido
df_marzo_final = df_completo[df_completo['fecha'].dt.month == 3].copy()

In [14]:
# Calcular promedio de rotación mensual
df_marzo_final['promedio_rotacion_mensual'] = df_marzo_final.groupby('id_socio')['rotacion_diaria'].transform('mean')

In [15]:
# Reordenar columnas como pediste
columnas_ordenadas = [
    'id_socio', 
    'socio', 
    'fecha', 
    'pallets', 
    'cantidad_pallets_sacados', 
    'stock_ayer', 
    'ingresos_estimados', 
    'rotacion_diaria', 
    'promedio_rotacion_mensual'
]

In [16]:
ruta1 = 'C:/Users/Sistemas/Desktop/pallets_cruzados_nuevo/scripts/prueba.xlsx'
df_marzo_final.to_excel(ruta1, index=False)